In [1]:
import pandas as pd

import nltk
from nltk.corpus import stopwords
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation


In [3]:
posts_path = '4chan/output/posts.csv'
threads_path = '4chan/output/threads.csv'

chan4_posts = pd.read_csv(posts_path)
chan4_threads = pd.read_csv(threads_path)

posts_end_path  = 'endchan V2/output/posts.csv'
threads_end_path = 'endchan V2/output/threads.csv'

endchan_posts = pd.read_csv(posts_end_path)
endchan_threads = pd.read_csv(threads_end_path)

In [4]:
chan4_posts.head(5)
# 40459 x 4

,subpost_id,content,time,thread_id
0,p501099183,>>501098958 (OP)I used an anal egg to make me ...,03/21/25(Fri)13:43:02No.501099183,p501098958
1,p501099437,corporate farming was a mistakepicrel is the c...,03/21/25(Fri)13:46:36No.501099437,p501098958
2,p501100092,>>501098958 (OP)Almost like the old administra...,03/21/25(Fri)13:55:59No.501100092,p501098958
3,p501100224,">>501100092Trump started the fake pandemic, st...",03/21/25(Fri)13:57:44No.501100224,p501098958
4,p501101758,>>501098958 (OP)>what did americans mean by th...,03/21/25(Fri)14:17:04No.501101758,p501098958


In [5]:
chan4_threads.head(5)
# 470 x 6

,thread_id,thread_text,time
0,p501098958,>kill all of our domestic chickens because fak...,03/21/25(Fri)13:40:04No.501098958
1,p501104660,Denmark is the best country in Europe and I'm ...,03/21/25(Fri)14:53:50No.501104660
2,p501098958,>kill all of our domestic chickens because fak...,03/21/25(Fri)13:40:04No.501098958
3,p501101640,NaN,03/21/25(Fri)14:15:39No.501101640
4,p501099426,You can't be racist and pagan.You can't be pro...,03/21/25(Fri)13:46:30No.501099426


Already noticing the time is off, so will have to clean this up

In [6]:
endchan_posts.head(5)
# 6014 x 4

,subpost_id,content,time,thread_id
0,91434,>>91417>killed off most of the savage Natives ...,10/28/2023 (Sat) 08:22,91417
1,91435,Netanyahu's not a white boy. Nor is any Yidd. ...,10/28/2023 (Sat) 11:34,91417
2,91448,NaN,10/29/2023 (Sun) 14:19,91417
3,92023,Idk but it makes me want to pull my hair out. ...,12/26/2023 (Tue) 02:46,91417
4,92024,>>92023,12/26/2023 (Tue) 07:50,91417


In [7]:
endchan_threads.head(5)
# 135 x 3

,thread_id,thread_text,time
0,91417,"What's wrong with all of you poor, pathetic, s...",10/27/2023 (Fri) 15:50
1,95327,1) disable or heavily damage the kike propagan...,12/08/2024 (Sun) 23:36
2,93932,They like to call Jerry Seinfeld a “funnyman” ...,06/15/2024 (Sat) 11:36
3,90261,The Natashas 2003.https://www.scribd.com/docum...,04/14/2023 (Fri) 02:05
4,94806,Hello from soyjak.party.gemmy board,09/28/2024 (Sat) 02:57


# Topic modeling with 4chan data

Thread is the main post, posts is the comments underneath one thread <br>
Topics across one thread should be the same, as the thread decides the topic. Therefore, it is most logical to use threads to do topic modeling.

In [8]:
# Cleaning the time column of chan4_threads
chan4_threads['time'] = chan4_threads['time'].str.split('No.').str[0]

## Basic topic modeling with regular libraries

### NLTK and Scikit-Learn

In [9]:
my_stopwords = nltk.corpus.stopwords.words('english')
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'

In [10]:
# cleaning master function
def clean_thread(thread, bigrams=False):
    if not isinstance(thread, str):  # Handle non-string values
        thread = ''
    thread = thread.lower()  # lower case
    thread = re.sub('[' + my_punctuation + ']+', ' ', thread)  # strip punctuation
    # thread = re.sub('\s+', ' ', thread)  # remove double spacing
    thread = re.sub('([0-9]+)', '', thread)  # remove numbers
    thread_token_list = [word for word in thread.split(' ')
                         if word not in my_stopwords]  # remove stopwords

    # thread_token_list = [word_rooter(word) if '#' not in word else word
    #                      for word in thread_token_list]  # apply word rooter
    if bigrams:
        thread_token_list = thread_token_list + [thread_token_list[i] + '_' + thread_token_list[i + 1]
                                                 for i in range(len(thread_token_list) - 1)]
    thread = ' '.join(thread_token_list)
    return thread


chan4_threads['cleaned'] = chan4_threads['thread_text'].apply(clean_thread)
chan4_threads['cleaned']

0       kill domestic chickens fake pcr test said egg...
1          denmark best country europe tired pretending 
2       kill domestic chickens fake pcr test said egg...
3                                                       
4      racist pagan pro white pagan pro christian pag...
                             ...                        
465     white east asian interbreeding le bad princip...
466    docsits relevantisraeli nuclear program spotte...
467    young white men across world turning fascism n...
468    libs slated get majority  accelerationism answ...
469     baltics   whitebaltics   white baltics   whit...
Name: cleaned, Length: 470, dtype: object

In [11]:

# the vectorizer object will be used to transform text to vector form
vectorizer = CountVectorizer(max_df=0.9, min_df=25, token_pattern='\w+|\$[\d\.]+|\S+')

# apply transformation
tf = vectorizer.fit_transform(chan4_threads['cleaned']).toarray()

# tf_feature_names tells us what word each column in the matric represents
tf_feature_names = vectorizer.get_feature_names_out()

In [12]:
number_of_topics = 5
model = LatentDirichletAllocation(n_components=number_of_topics, random_state=0)
model.fit(tf)

LatentDirichletAllocation(n_components=5, random_state=0)

In [13]:
def display_topics(model, feature_names, no_top_words):
    topic_dict = {}
    for topic_idx, topic in enumerate(model.components_):
        topic_dict["Topic %d words" % (topic_idx)]= ['{}'.format(feature_names[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        topic_dict["Topic %d weights" % (topic_idx)]= ['{:.1f}'.format(topic[i])
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
    return pd.DataFrame(topic_dict)

In [14]:
no_top_words = 15
display_topics(model, tf_feature_names, no_top_words)

,Topic 0 words,Topic 0 weights,Topic 1 words,Topic 1 weights,Topic 2 words,Topic 2 weights,Topic 3 words,Topic 3 weights,Topic 4 words,Topic 4 weights
0,https,128.2,us,42.1,make,70.4,even,39.5,white,78.5
1,trump,120.0,go,35.0,like,45.9,www,36.0,would,61.3
2,com,101.2,europe,30.2,get,23.1,uk,33.2,people,36.1
3,www,55.4,’s,26.4,year,18.4,new,26.2,world,29.7
4,year,14.0,like,18.5,would,7.3,’s,19.8,literally,27.2
5,every,10.4,uk,7.3,people,6.7,full,18.5,government,14.7
6,uk,9.1,world,5.7,white,0.9,go,15.4,even,9.9
7,make,5.3,www,0.2,even,0.2,people,12.9,every,9.5
8,full,4.9,year,0.2,go,0.2,make,11.9,get,8.3
9,government,4.8,would,0.2,government,0.2,government,8.1,full,4.2


### BERTopic

In [15]:
from bertopic import BERTopic

d:\OneDrive - TU Eindhoven\JADS\Semester B\Data Forensics\Project\Data-Forensics\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
topic_model = BERTopic()
docs = chan4_threads['cleaned'].tolist()
topics, probs = topic_model.fit_transform(docs)
chan4_threads['topics'] = topics
chan4_threads['topic_probs'] = probs

In [17]:
BERTopics = topic_model.get_topic_info()
BERTopics

,Topic,Count,Name,Representation,Representative_Docs
0,-1,17,-1_india_china_manufacturing_chinese,"[india, china, manufacturing, chinese, factori...",[india failed attract factories china went wr...
1,0,37,0_white_search_google_anti,"[white, search, google, anti, nobody, men, fam...",[objectively nasty world class rbf tds nut f...
2,1,36,1_recall_european_would_tesla,"[recall, european, would, tesla, glue, musk, e...",[morons using wrong glue tesla forced recall e...
3,2,34,2_notice_courts_something_libs,"[notice, courts, something, libs, slated, acce...",[libs slated get majority accelerationism ans...
4,3,29,3_faces_sold_want_even,"[faces, sold, want, even, child, friends, make...",[coworker mental breakdown yesterday work fire...
5,4,28,4_make_would_woods_christianity,"[make, would, woods, christianity, us, live, b...",[protestantism sort anti religion course exis...
6,5,26,5_care_baltics_fuck_russians,"[care, baltics, fuck, russians, whitebaltics, ...",[proud ukrainians inflicting much pain sufferi...
7,6,25,6_ukraine_bbc_uk_catbox,"[ukraine, bbc, uk, catbox, moe, news, co, arti...",[previous →day — daily battlefield assessme...
8,7,23,7_step_great_every_literal,"[step, great, every, literal, america, assassi...",[antifa violence endless new waves vandalism ...
9,8,22,8_trump_https_com_washington,"[trump, https, com, washington, dc, youtu, pre...",[president donald j trump potushttps www dona...


In [18]:
topic_model.visualize_topics()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([[0, 'white | search | google | anti | nobody', 37],
                                   [1, 'recall | european | would | tesla | glue', 36],
                                   [2, 'notice | courts | something | libs | slated', 34],
                                   [3, 'faces | sold | want | even | child', 29],
                                   [4, 'make | would | woods | christianity | us', 28],
                                   [5, 'care | baltics | fuck | russians | whitebaltics', 26],
                                   [6, 'ukraine | bbc | uk | catbox | moe', 25],
                                   [7, 'step | great | every | literal | america', 23],
                                   [8, 'trump | https | com | washington | dc', 22],
                                   [9, 'poilievre | polls | policy | real | never', 22],
                                   [10, 'convinced | shitalians | tomatokikes | semitic | majority', 21],
                                   [11, 'go | coffee | br | woke | drink', 21],
                                   [12, 'prison | lockdown | inmates | staff | thomson', 21],
                                   [13, 'chickens | eggs | test | genocide | pro', 18],
                                   [14, 'niggers | dei | jobs | use | hate', 17],
                                   [15, 'pagan | post | nazi | memes | pro', 15],
                                   [16, 'english | dropped | recognize | official | wqygcbceceu', 13],
                                   [17, 'consciousness | soul | its | like | dwelled', 12],
                                   [18, 'netanyahu | palestine | backing | base | conspiracy', 11],
                                   [19, 'us | commonwealth | americas | offer | politics', 11],
                                   [20, 'east | white | bunnies | snow | interbreeding', 11]], dtype=object),
              'hovertemplate': '<b>Topic %{customdata[0]}</b><br>%{customdata[1]}<br>Size: %{customdata[2]}',
              'legendgroup': '',
              'marker': {'color': '#B0BEC5',
                         'line': {'color': 'DarkSlateGrey', 'width': 2},
                         'size': {'bdata': 'JSQiHRwaGRcWFhUVFRIRDw0MCwsL', 'dtype': 'i1'},
                         'sizemode': 'area',
                         'sizeref': 0.023125,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': {'bdata': ('/xePQGZOzUCRA45AnU+NviB9rECh07' ... 'i2QHrSlUGm4p5AEFPvQLPrlkHhG51A'),
                    'dtype': 'f4'},
              'xaxis': 'x',
              'y': {'bdata': ('VMRVQABCkUCAPnlB12KiwFjyeUAz1Y' ... 'xiQCJRsEGU0HBB3WbFQBJtsUErLUlA'),
                    'dtype': 'f4'},
              'yaxis': 'y'}],
    'layout': {'annotations': [{'showarrow': False,
                                'text': 'D1',
                                'x': np.float32(-0.45671436),
                                'y': np.float32(9.764962),
                                'yshift': 10},
                               {'showarrow': False,
                                'text': 'D2',
                                'x': np.float32(10.619068),
                                'xshift': 10,
                                'y': np.float32(25.504995)}],
               'height': 650,
               'hoverlabel': {'bgcolor': 'white', 'font': {'family': 'Rockwell', 'size': 16}},
               'legend': {'itemsizing': 'constant', 'tracegroupgap': 0},
               'margin': {'t': 60},
               'shapes': [{'line': {'color': '#CFD8DC', 'width': 2},
                           'type': 'line',
                           'x0': np.float32(10.619068),
                           'x1': np.float32(10.619068),
                           'y0': np.float32(-5.9750705),
                           'y1': np.float32(25.504995

## DarkBERT

In [19]:
from transformers import pipeline

In [20]:
with open('acces_token.txt', 'r') as file:
    access_token = file.read().strip()

In [21]:
unmasker = pipeline('fill-mask', model="s2w-ai/DarkBERT", token=access_token)
unmasker("RagnarLocker, LockBit, and REvil are types of <mask>.")

Device set to use cpu


[{'score': 0.49523264169692993,
  'token': 25346,
  'token_str': ' ransomware',
  'sequence': 'RagnarLocker, LockBit, and REvil are types of ransomware.'},
 {'score': 0.04661604389548302,
  'token': 16886,
  'token_str': ' malware',
  'sequence': 'RagnarLocker, LockBit, and REvil are types of malware.'},
 {'score': 0.042176585644483566,
  'token': 28811,
  'token_str': ' wallets',
  'sequence': 'RagnarLocker, LockBit, and REvil are types of wallets.'},
 {'score': 0.02898237481713295,
  'token': 2196,
  'token_str': ' drugs',
  'sequence': 'RagnarLocker, LockBit, and REvil are types of drugs.'},
 {'score': 0.020001336932182312,
  'token': 11344,
  'token_str': ' hackers',
  'sequence': 'RagnarLocker, LockBit, and REvil are types of hackers.'}]

In [22]:
# Iterate through the 'Representation' column in BERTopics DataFrame
for representation in BERTopics['Representation']:
    # Take only the first 3 words from the representation list
    words = ', '.join(representation[:6])
    # Create the input sentence for the unmasker
    sentence = f"{words} are all types of <mask>"
    # Use the unmasker to predict the masked word
    result = unmasker(sentence)
    print(f"Input: {sentence}")
    print(f"Prediction: {result}\n")

Input: india, china, manufacturing, chinese, factories, threat are all types of <mask>
Prediction: [{'score': 0.21873830258846283, 'token': 3455, 'token_str': ' threats', 'sequence': 'india, china, manufacturing, chinese, factories, threat are all types of threats'}, {'score': 0.17531301081180573, 'token': 1856, 'token_str': ' threat', 'sequence': 'india, china, manufacturing, chinese, factories, threat are all types of threat'}, {'score': 0.02046685852110386, 'token': 4854, 'token_str': ' danger', 'sequence': 'india, china, manufacturing, chinese, factories, threat are all types of danger'}, {'score': 0.019146598875522614, 'token': 383, 'token_str': ' things', 'sequence': 'india, china, manufacturing, chinese, factories, threat are all types of things'}, {'score': 0.0177216287702322, 'token': 4, 'token_str': '.', 'sequence': 'india, china, manufacturing, chinese, factories, threat are all types of.'}]

Input: white, search, google, anti, nobody, men are all types of <mask>
Prediction: